In [1]:
# Outputs group file for students with greater than a 2.0 GPA

import pandas as pd
import os
from datetime import date, datetime
from sqlalchemy import create_engine

begin_academic_year = '2011'

# local connection information
db_user = os.environ.get('DB_USER')
db_pass = os.environ.get('DB_PASS')
engine = create_engine(f'mssql+pyodbc://{db_user}:{db_pass}' +
                       '@PSC-SQLProd/Campus6?' +
                       'driver=ODBC+Driver+13+for+SQL+Server')
connection = engine.connect()

sql_str = """
SELECT [PEOPLE_CODE_ID]
      ,[ACADEMIC_YEAR]
      ,[ACADEMIC_TERM]
      ,[ACADEMIC_SESSION]
      ,[RECORD_TYPE]
      ,[GPA]
FROM [Campus6].[dbo].[TRANSCRIPTGPA]
"""
df = pd.read_sql_query(sql_str, connection)

df = df[df['ACADEMIC_YEAR'] >= begin_academic_year]

# Rename people_code_id to student_integration_id
df = df.rename(columns={'PEOPLE_CODE_ID': 'student_integration_id' })

# filter results to only have cumulative GPA's equal to or above a 2.0,
#                   have a record type of 'O'
df = df[df['GPA'] >= 2]
df = df[df['RECORD_TYPE'] == 'O']
df = df[df['ACADEMIC_TERM'].isin(['SPRING', 'SUMMER', 'FALL'])]

print(df.shape)

(15693, 6)


In [2]:
df['ACADEMIC_YEAR'] = (pd.to_numeric(df['ACADEMIC_YEAR'], errors='coerce') )

In [3]:
df_seq = pd.DataFrame([{'term': 'SPRING', 'seq': 1},
                       {'term': 'SUMMER', 'seq': 2},
                       {'term': 'FALL', 'seq': 3}])
print(df_seq)
df = pd.merge(df, df_seq, left_on='ACADEMIC_TERM', right_on='term', how='left')
print(df.head())
df['term_seq'] =  df['ACADEMIC_YEAR']*100 + df['seq']
print(df.head())

   seq    term
0    1  SPRING
1    2  SUMMER
2    3    FALL
  student_integration_id  ACADEMIC_YEAR ACADEMIC_TERM ACADEMIC_SESSION  \
0             P000000006           2011        SUMMER                    
1             P000000030           2012          FALL                    
2             P000000597           2011          FALL                    
3             P000000597           2011        SPRING                    
4             P000000597           2011        SUMMER                    

  RECORD_TYPE   GPA  seq    term  
0           O  4.00    2  SUMMER  
1           O  4.00    3    FALL  
2           O  3.18    3    FALL  
3           O  3.20    1  SPRING  
4           O  3.20    2  SUMMER  
  student_integration_id  ACADEMIC_YEAR ACADEMIC_TERM ACADEMIC_SESSION  \
0             P000000006           2011        SUMMER                    
1             P000000030           2012          FALL                    
2             P000000597           2011          FALL          

In [4]:
df.reset_index().groupby(['student_integration_id'])['term_seq'].max()

student_integration_id
P000000006    201102
P000000030    201203
P000000597    201301
P000001163    201101
P000001329    201202
P000001518    201101
P000001770    201103
P000002356    201101
P000002590    201101
P000003107    201703
P000003471    201501
P000003938    201101
P000004142    201301
P000004357    201103
P000004567    201502
P000005941    201203
P000005959    201101
P000006163    201101
P000006249    201101
P000006386    201101
P000006789    201101
P000006798    201502
P000006911    201202
P000006924    201101
P000007236    201103
P000007334    201102
P000008284    201101
P000008449    201101
P000008918    201103
P000008925    201103
               ...  
P000054277    201703
P000054281    201703
P000054307    201603
P000054318    201703
P000054354    201703
P000054375    201703
P000054390    201703
P000054391    201702
P000054419    201703
P000054424    201603
P000054426    201703
P000054427    201703
P000054429    201603
P000054430    201703
P000054433    201703
P000054435 

In [5]:
# find the latest year
df.loc[df.reset_index().groupby(['student_integration_id'])['term_seq'].idxmax()]


,student_integration_id,ACADEMIC_YEAR,ACADEMIC_TERM,ACADEMIC_SESSION,RECORD_TYPE,GPA,seq,term,term_seq
0,P000000006,2011,SUMMER,,O,4.00,2,SUMMER,201102
1,P000000030,2012,FALL,,O,4.00,3,FALL,201203
6,P000000597,2013,SPRING,,O,3.19,1,SPRING,201301
7,P000001163,2011,SPRING,,O,2.27,1,SPRING,201101
8,P000001329,2012,SUMMER,,O,3.46,2,SUMMER,201202
9,P000001518,2011,SPRING,,O,2.96,1,SPRING,201101
10,P000001770,2011,FALL,,O,2.25,3,FALL,201103
12,P000002356,2011,SPRING,,O,3.19,1,SPRING,201101
13,P000002590,2011,SPRING,,O,2.39,1,SPRING,201101
17,P000003107,2017,FALL,,O,3.61,3,FALL,201703


In [6]:
print(df.shape)

(15693, 9)


In [7]:
df = (df.loc[df.reset_index()
               .groupby(['student_integration_id'])['term_seq']
               .idxmax()])

In [8]:
print(df.shape)

(4915, 9)


In [9]:
# create prereq group identifier
df['prereq_group_identifier'] = 'GPA_GT_2.0'

In [10]:
# columns to keep
df = df.loc[:, ['student_integration_id', 'prereq_group_identifier']]

In [11]:
df = df.drop_duplicates(['student_integration_id'], keep='first')
print(df.shape)

(4915, 2)


In [12]:
today = datetime.now().strftime('%Y%m%d')
fn_output = today + '_student_higherthan20gpa_student_prereq_groups.txt'
df.to_csv(fn_output, index=False)